In [1]:
import pandas as pd
import numpy as np

In [2]:
# Function to make 4 classifications based on balance due and the portion of the penality paid
def make_classification(penality, paid, due):
    if due == 0:
        if penality > paid:
            classification = 0
            desc = 'NO BALANCE - PARITALLY PAID'
        else:
            classification = 1
            desc = 'NO BALANCE - PAID IN FULL'
    else:
        if paid == 0:
            classification = 2
            desc = 'BALANCE DUE - UNPAID'
        else:
            classification = 3
            desc = 'BALANCE DUE - PARITALLY PAID'
    return classification, desc

# convert values to float
def make_float(expected_float):
    try:
        if type(expected_float) == str:
            expected_float = expected_float.replace('+', '').replace(',', '')
        return float(expected_float)
    except:
        # print expected_float
        return np.nan

In [2]:
# Trump owned buildings identified from:
# https://www.bisnow.com/new-york/news/economy/how-big-is-trumps-nyc-empire-63995?single-page

# BIN NUMBER
_40_WALL_ST = '1001018' # TRUMP_BUILDING
_1290_AVENUE_OF_THE_AMERICAS = '1034510' # _1290_AVENUE_OF_THE_AMERICAS 30% Stake in Vornado owned building
_1_CENTRAL_PARK_WEST = '1027191' # TRUMP_INTERNATIONAL_HOTEL_AND_TOWER
_725_5TH_AVE = '1035794' # TRUMP_TOWER
_246_SPRING_ST = '1088431' # TRUMP_SOHO
_502_PARK_AVE = '1040756' # TRUMP_PARK_AVE
_327_E_47TH_ST = '1038908' # TRUMP_WORLD_TOWER
_200_E_69TH_ST = '1043902' # TRUMP_PALACE
_106_CENTRAL_PARK_S = '1069595' # TRUMP_PARC
_610_PARK_AVE = '1041086' # _610_PARK_AVE

TRUMP_BUILDINGS = [_106_CENTRAL_PARK_S, _1290_AVENUE_OF_THE_AMERICAS, 
                   _1_CENTRAL_PARK_WEST, _200_E_69TH_ST, _246_SPRING_ST, 
                   _327_E_47TH_ST, _40_WALL_ST, _502_PARK_AVE,
                   _610_PARK_AVE, _725_5TH_AVE]
# Wollman Rink - Central Park (Not a building)

In [31]:
# DOB ECB BUILDING VIOLATION DATA
DOB_ECB = 'data/20170322_DOB_ECB_Violations.csv'
ACS_5YR_RACE = 'data/CENSUS_TRACT_RACE_INCOME/ACS_15_5YR_DP05_with_ann.csv'
ACS_5YR_INCOME = 'data/CENSUS_TRACT_RACE_INCOME/ACS_15_5YR_S1901_with_ann.csv'

In [4]:
DF_DOB_ECB = pd.read_csv(DOB_ECB, usecols=['BIN', 'ISSUE_DATE', 'SEVERITY', 
                                           'PENALITY_IMPOSED', 'AMOUNT_PAID', 
                                           'BALANCE_DUE', 'ECB_VIOLATION_STATUS', 
                                           'BORO', 'BLOCK', 'LOT', 'VIOLATION_DESCRIPTION',
                                           'INFRACTION_CODE1'],
                         dtype={'BIN': str, 'PENALITY_IMPOSED': float,
                                'AMOUNT_PAID': float, 'BALANCE_DUE': float}) 

/Users/Nate/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Only 181 FINES associated with trump buildings

print '"TRUMP PROPERTIES" WITH FINES', len(DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))])
print 'PENALITY IMPOSED', DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))]['PENALITY_IMPOSED'].sum()
print 'AMOUNT_PAID', DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))]['AMOUNT_PAID'].sum()
print 'OUTSTANDING BALANCE', DF_DOB_ECB[(DF_DOB_ECB['PENALITY_IMPOSED'] > 0) & (DF_DOB_ECB['BIN'].isin(TRUMP_BUILDINGS))]['BALANCE_DUE'].sum()

"TRUMP PROPERTIES" WITH FINES 181
PENALITY IMPOSED 183425.0
AMOUNT_PAID 177357.27
OUTSTANDING BALANCE 765.34


## Perhaps fines associated with trump owned buildings may not hold enough data

In [6]:
# VIOLATIONS WITH A BALANCE DUE
DF_DOB_ECB[DF_DOB_ECB['BALANCE_DUE'] > 0]['BALANCE_DUE'].sum()

748325710.25

In [7]:
# TOTAL PENALITIES VALUE
DF_DOB_ECB['PENALITY_IMPOSED'].sum()

1669745657.52

In [8]:
# FINES PAID
DF_DOB_ECB['AMOUNT_PAID'].sum()

616422508.6099999

In [10]:
DF_DOB_ECB_FINES = DF_DOB_ECB[DF_DOB_ECB['PENALITY_IMPOSED'] > 0].copy()

In [12]:
DF_DOB_ECB_FINES['CLASSIFICATION'], DF_DOB_ECB_FINES['CLASS_DESC'] = np.vectorize(make_classification)(DF_DOB_ECB_FINES['PENALITY_IMPOSED'],
                                                                                                       DF_DOB_ECB_FINES['AMOUNT_PAID'],
                                                                                                       DF_DOB_ECB_FINES['BALANCE_DUE'])

In [13]:
# Breakdown of different classes
DF_DOB_ECB_FINES[['CLASS_DESC', 'ECB_VIOLATION_STATUS']].groupby('CLASS_DESC').count()

,ECB_VIOLATION_STATUS
CLASS_DESC,
BALANCE DUE - PARITALLY PAID,31699
BALANCE DUE - UNPAID,104709
NO BALANCE - PAID IN FULL,479941
NO BALANCE - PARITALLY PAID,136943


In [33]:
# race data
ACS_5YR_RACE_DF = pd.read_csv(ACS_5YR_RACE,skiprows=[1], usecols=['GEO.id2', 'HC01_VC03', 'HC01_VC49', 'HC01_VC50', 'HC01_VC51','HC01_VC56', 'HC01_VC64', 'HC01_VC69', 'HC01_VC23'])

# rename columns
ACS_5YR_RACE_DF.rename(columns={'HC01_VC03': 'TOTAL_POPULATION', 'HC01_VC49': 'WHITE',
                        'HC01_VC50': 'BLACK_AFRICAN_AMERICAN', 'HC01_VC51': 'AMERICAN_INDIAN_AND_ALASKA_NATIVE',
                        'HC01_VC56': 'ASIAN', 'HC01_VC64': 'NATIVE_HAWAIIAN_AND_OTHER_PACIFIC_ISLANDER',
                        'HC01_VC69': 'SOME_OTHER_RACE', 'HC01_VC23': 'MEDIAN_AGE', 'GEO.id2': 'GEOID'}, inplace=True)

for i in ACS_5YR_RACE_DF.columns[ACS_5YR_RACE_DF.columns!='GEOID']:
    ACS_5YR_RACE_DF[i] = ACS_5YR_RACE_DF[i].apply(lambda x: make_float(x))

# income data
ACS_5YR_INCOME_DF = pd.read_csv(ACS_5YR_INCOME,skiprows=[1],usecols=['GEO.id2', 'HC01_EST_VC01', 'HC01_EST_VC15'])

# rename columns
ACS_5YR_INCOME_DF.rename(columns={'HC01_EST_VC01': 'TOTAL_HOUSEHOLDS', 
                       'HC01_EST_VC15': 'MEAN_INCOME', 'GEO.id2': 'GEOID'}, inplace=True)

#convert values to float
for i in ACS_5YR_INCOME_DF.columns[ACS_5YR_INCOME_DF.columns!='GEOID']:
    ACS_5YR_INCOME_DF[i] = ACS_5YR_INCOME_DF[i].apply(lambda x: make_float(x))

In [35]:
BK = gp.read_file(PLUTO_BK)


,GEOID,TOTAL_POPULATION,MEDIAN_AGE,WHITE,BLACK_AFRICAN_AMERICAN,AMERICAN_INDIAN_AND_ALASKA_NATIVE,ASIAN,NATIVE_HAWAIIAN_AND_OTHER_PACIFIC_ISLANDER,SOME_OTHER_RACE
0,36001000100,2001.0,33.8,503.0,1229.0,0.0,48.0,0.0,6.0
1,36001000200,4519.0,32.9,557.0,3750.0,0.0,13.0,0.0,39.0
2,36001000300,5244.0,34.2,2541.0,1965.0,14.0,135.0,0.0,149.0
3,36001000401,2423.0,61.3,2210.0,101.0,46.0,66.0,0.0,0.0
4,36001000403,4654.0,34.1,3247.0,741.0,37.0,350.0,0.0,160.0
5,36001000404,5033.0,19.7,3441.0,805.0,0.0,401.0,3.0,168.0
6,36001000501,3758.0,30.8,2138.0,983.0,14.0,236.0,0.0,119.0
7,36001000502,3747.0,21.3,2673.0,405.0,0.0,346.0,6.0,101.0
8,36001000600,3851.0,24.2,1372.0,1654.0,19.0,310.0,0.0,376.0
9,36001000700,3927.0,29.0,559.0,3139.0,0.0,17.0,0.0,45.0
